In [2]:
import sys
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))
import pandas as pd
from src.data_io import load_onspd, load_census
from src.features import add_econ_percentage

DATA_DIR = Path.home()/"/Users/aleksanderskutnik/Library/Mobile Documents/com~apple~CloudDocs/databases"
RAW_DIR = DATA_DIR/"raw"
PROCESSED_DIR = DATA_DIR/"processed"
onspd_path = RAW_DIR/"onspd.csv"
census_path = RAW_DIR/"econactivity.csv"
#load datasets
onspd = load_onspd(onspd_path)
onspd = onspd[onspd["oa21"].astype(str).str.startswith("E")].copy()
census = load_census(census_path)
display(onspd.head())
display(census.head())
#drop rows where oa21 is missing
census = census[census["oa21"].notna()].copy()
#main numeric column cleaning steps
total_col = "Total: All usual residents aged 16 years and over"
census[total_col] = pd.to_numeric(census[total_col],errors="coerce")
#drop rows where the total is NaN
census = census[census[total_col].notna()].copy()

/Users/aleksanderskutnik/ecs7019u-group4/src/data_io.py:12: DtypeWarning: Columns (50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path,usecols=usecols)


,pcd,pcds,lat,long,imd,oa21,lsoa21,msoa21
39259,AL1 1AA,AL1 1AA,51.749084,-0.341337,30564,E00120568,E01023743,E02004937
39260,AL1 1AB,AL1 1AB,51.747828,-0.301865,30628,E00120196,E01023667,E02004936
39261,AL1 1AD,AL1 1AD,51.747828,-0.301865,30628,E00120196,E01023667,E02004936
39262,AL1 1AE,AL1 1AE,51.747828,-0.301865,30628,E00120196,E01023667,E02004936
39263,AL1 1AF,AL1 1AF,51.749084,-0.341337,30564,E00120568,E01023743,E02004937


,oa21,mnemonic,Total: All usual residents aged 16 years and over,Economically active (excluding full-time students),Economically active (excluding full-time students):In employment,Economically active (excluding full-time students):In employment:Employee,Economically active (excluding full-time students): In employment: Employee: Part-time,Economically active (excluding full-time students): In employment: Employee: Full-time,Economically active (excluding full-time students):In employment:Self-employed with employees,Economically active (excluding full-time students): In employment: Self-employed with employees: Part-time,...,Economically active (excluding full-time students):In employment:Self-employed without employees,Economically active (excluding full-time students): In employment: Self-employed without employees: Part-time,Economically active (excluding full-time students): In employment: Self-employed without employees: Full-time,Economically active (excluding full-time students): Unemployed,Economically inactive,Economically inactive: Retired,Economically inactive: Student,Economically inactive: Looking after home or family,Economically inactive: Long-term sick or disabled,Economically inactive: Other
0,E00000001,E00000001,161.0,98.0,88.0,61.0,17.0,44.0,8.0,1.0,...,19.0,10.0,9.0,10.0,63.0,52.0,5.0,2.0,1.0,3.0
1,E00000003,E00000003,223.0,139.0,134.0,90.0,17.0,73.0,10.0,1.0,...,34.0,18.0,16.0,5.0,83.0,57.0,19.0,6.0,0.0,1.0
2,E00000005,E00000005,105.0,68.0,65.0,45.0,7.0,38.0,6.0,0.0,...,14.0,4.0,10.0,3.0,36.0,23.0,3.0,3.0,1.0,6.0
3,E00000007,E00000007,142.0,110.0,109.0,94.0,7.0,87.0,1.0,0.0,...,14.0,8.0,6.0,1.0,32.0,17.0,11.0,1.0,1.0,2.0
4,E00000010,E00000010,170.0,128.0,110.0,77.0,11.0,66.0,3.0,0.0,...,30.0,15.0,15.0,18.0,41.0,15.0,8.0,0.0,9.0,9.0


In [3]:
#join onspd with census data
merged = onspd.merge(
    census,
    on="oa21",
    how="left",
    validate="m:1"
)
print(merged.shape)
merged.head()

(2255230, 28)


,pcd,pcds,lat,long,imd,oa21,lsoa21,msoa21,mnemonic,Total: All usual residents aged 16 years and over,...,Economically active (excluding full-time students):In employment:Self-employed without employees,Economically active (excluding full-time students): In employment: Self-employed without employees: Part-time,Economically active (excluding full-time students): In employment: Self-employed without employees: Full-time,Economically active (excluding full-time students): Unemployed,Economically inactive,Economically inactive: Retired,Economically inactive: Student,Economically inactive: Looking after home or family,Economically inactive: Long-term sick or disabled,Economically inactive: Other
0,AL1 1AA,AL1 1AA,51.749084,-0.341337,30564,E00120568,E01023743,E02004937,E00120568,270.0,...,21.0,11.0,10.0,6.0,70.0,42.0,8.0,13.0,2.0,5.0
1,AL1 1AB,AL1 1AB,51.747828,-0.301865,30628,E00120196,E01023667,E02004936,E00120196,286.0,...,23.0,10.0,13.0,5.0,83.0,55.0,12.0,7.0,5.0,4.0
2,AL1 1AD,AL1 1AD,51.747828,-0.301865,30628,E00120196,E01023667,E02004936,E00120196,286.0,...,23.0,10.0,13.0,5.0,83.0,55.0,12.0,7.0,5.0,4.0
3,AL1 1AE,AL1 1AE,51.747828,-0.301865,30628,E00120196,E01023667,E02004936,E00120196,286.0,...,23.0,10.0,13.0,5.0,83.0,55.0,12.0,7.0,5.0,4.0
4,AL1 1AF,AL1 1AF,51.749084,-0.341337,30564,E00120568,E01023743,E02004937,E00120568,270.0,...,21.0,11.0,10.0,6.0,70.0,42.0,8.0,13.0,2.0,5.0


In [4]:
#add percentage features
features_oa = add_econ_percentage(merged)
keep_cols = [
    "oa21",
    "pcds",
    "lat",
    "long",
    "imd",
    "total16plus",
    "pct_econ_active",
    "pct_econ_inactive",
    "pct_retired",
    "pct_students",
    "pct_home_family",
    "pct_long_term_sick",
    "pct_unemployed",
    "pct_self_employed",
]
features_oa = features_oa[keep_cols]
features_oa.head()

,oa21,pcds,lat,long,imd,total16plus,pct_econ_active,pct_econ_inactive,pct_retired,pct_students,pct_home_family,pct_long_term_sick,pct_unemployed,pct_self_employed
0,E00120568,AL1 1AA,51.749084,-0.341337,30564,270.0,71.851852,25.925926,15.555556,2.962963,4.814815,0.740741,2.222222,10.000000
1,E00120196,AL1 1AB,51.747828,-0.301865,30628,286.0,68.881119,29.020979,19.230769,4.195804,2.447552,1.748252,1.748252,10.839161
2,E00120196,AL1 1AD,51.747828,-0.301865,30628,286.0,68.881119,29.020979,19.230769,4.195804,2.447552,1.748252,1.748252,10.839161
3,E00120196,AL1 1AE,51.747828,-0.301865,30628,286.0,68.881119,29.020979,19.230769,4.195804,2.447552,1.748252,1.748252,10.839161
4,E00120568,AL1 1AF,51.749084,-0.341337,30564,270.0,71.851852,25.925926,15.555556,2.962963,4.814815,0.740741,2.222222,10.000000


In [6]:
#save feature table
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_DIR/"features_by_oa.csv"
features_oa.to_csv(out_path, index=False)